# S1 - Federated Learning for IoT and Smart Cities

## Code Structure

1. Setup enviroment by importing libs and dataset
2. Preprocess the data by
- Flatten the data
- Selects clients (if needed)
- Batch and create dict
3. Prepare the model
- Keras model 
- Create layers
- Cast keras model to tff-model
4. Build and test FL algorithm
- Initialization of model on server.
Then as a "loop" :
- server-to-client broadcast step.
- local client update step.
- client-to-server upload step.
- server update step

### 1. Setup enviroment

In [1]:
import nest_asyncio
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff # this might register as non-resolved but seems to work
np.random.seed(0)
nest_asyncio.apply()
# check if tff works correctly
tff.federated_computation(lambda: 'Hello, World!')()

2021-11-30 10:53:16.883322: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-30 10:53:16.884137: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-30 10:53:26.998137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-30 10:53:26.998469: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-30 10:53:27.000143: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-5B2NUIL3): /proc/driver/nvidia/version does not exist
2021-11-30 10:53:27.007118: I tensorflow/core/platform/cpu_fe

b'Hello, World!'

In [2]:
#Load the EMNIST data set
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

### 2. Process the data

Flatten batches of data, and conver to a tuple (flattened_image_vector, label)

In [3]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

Select a small subset of clients, and apply the above created preprocessing to them

In [4]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## 3. Preparing the model

Create a keras ML model, with one hidden layer and a softmax layer

In [7]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  #Create keras model, and return
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

Wrap this model as a 'tff.learning.Model' for use in Tensorflow Federated

In [8]:
def model_fn():
  #Call on function for creating a keras moedl
  keras_model = create_keras_model()
  
  #Convert to TFF model, and return
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## 4. Building our own FL algorithm

Create functions for initializing the server (initialize_fn), as well as performing one round of communication in the FL algorithm (next_fn)

In [9]:
def initialize_fn():
  #Call on function to create model
  model = model_fn()
  #Return the trainable weights
  return model.trainable_variables

In [10]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

### 4.2 Client update step

Gradient descent(?)

In [13]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y), client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

### 4.4 Server update step

A simple "FedAvd" update (aka "vanilla FL")

In [14]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y), model_weights, mean_client_weights)
  return model_weights